In [ ]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install gradio
!pip install pillow

In [1]:
import cv2 as cv
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

from PIL import Image
from utils import Cam, Img, IO

In [31]:
class Render:
    @staticmethod
    def draw_coordinate_frame(image_points, img):
        x0, y0 = image_points[:,0].astype(int)
        cv2.circle(img, (x0, y0), 9, (0, 0, 0), -1)

        x1, y1 = image_points[:,1].astype(int)
        img = cv2.arrowedLine(img, (x0, y0), (x1, y1), (255, 0, 0), 5)

        x2, y2 = image_points[:,2].astype(int)
        img = cv2.arrowedLine(img, (x0, y0), (x2, y2), (0, 255, 0), 5)

        x3, y3 = image_points[:,3].astype(int)
        img = cv2.arrowedLine(img, (x0, y0), (x3, y3), (0, 0, 255), 5)
        
    @staticmethod
    def image_axis():
        return
    @staticmethod
    def image_cal_marks():
        return

# Main calibration workflow
class CalibrationWorkflow:
    """Main calibration workflow using static helper classes"""
    
    def __init__(self, board_size=(9, 6), square_size=25.0, output_dir="/content/images"):
        self.board_size = board_size
        self.square_size = square_size
        self.output_dir = output_dir
        self.obj_points = []
        self.img_points = []
        self.processed_images = []
        self.calibration_results = None
    
    def process_images(self, files):
        """Process uploaded images and detect chessboard corners"""
        imgs = IO.get_images()
        if not files and not imgs:
            return "No files uploaded!"

        IO.store_images(files)
        self.imgs = IO.get_images()

        self.opoints, self.ipoints = Img.find_chessboard_corners(imgs)

        #IO.save_calibration_results([opoints, ipoints])
        """
        for images in imgs:
            ret, lam, dist, rvecs, tau = cv.calibrateCamera(opoints, ipoints, images.shape[::-1], None, None)

            rvecs = rvecs[0]
            omega = cv.Rodrigues(rvecs)
            omega = omega[0]
            tau = tau[0]
            print(omega)
        """
        return "Calibration Complete"
    
    def get_camera_pose_plot(self):
        """Get camera pose visualization"""
        if not self.calibration_results:
            return None
        
        return Render.create_camera_pose_plot(
            self.calibration_results['rvecs'],
            self.calibration_results['tvecs']
        )
    
    def get_sample_images_with_axes(self, num_samples=8):
        """Get sample images with coordinate axes drawn"""
        images = []
        for i in range(5):
            img = self.imgs[i]

            ret, lam, dist, rvecs, tau = cv.calibrateCamera(self.opoints, self.ipoints, img.shape[::-1], None, None)

            rvecs = rvecs[0]
            omega = cv.Rodrigues(rvecs)
            omega = omega[0]
            tau = tau[0]

            W = 2 * np.array([
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]
            ], dtype=np.float64)

            image_axes, jac = cv.projectPoints(W, rvecs, tau, lam, dist)

            image_axes = image_axes.squeeze().T

            x0, y0 = image_axes[:,0].astype(int)
            cv.circle(img, (x0, y0), 9, (0, 0, 0), -1)

            x1, y1 = image_axes[:,1].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x1, y1), (255, 0, 0), 5)

            x2, y2 = image_axes[:,2].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x2, y2), (0, 255, 0), 5)

            x3, y3 = image_axes[:,3].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x3, y3), (0, 0, 255), 5)

            pil = Image.fromarray(img)

            images.append(pil)

        return images[0], images[1], images[2], images[3], images[4]
    
    def get_undistortion_preview(self):
        """Get undistortion before/after comparison"""
        test_img = self.imgs[18]
        ret, lam, dist, rvecs, tau = cv.calibrateCamera(self.opoints, self.ipoints, test_img.shape[::-1], None, None)

        new_img = Img.undistort_image(test_img, lam, dist)

        original_pil = Image.fromarray(test_img)
        undistorted_pil = Image.fromarray(new_img)

        return undistorted_pil, undistorted_pil

In [30]:
# Create workflow instance
workflow = CalibrationWorkflow()

# Gradio interface
with gr.Blocks(title="Camera Calibration") as demo:
    gr.Markdown("#Camera Calibration Interface")
    gr.Markdown("Upload chessboard images and run calibration with visualization")
    
    with gr.Row():
        files = gr.File(file_count="multiple", file_types=[".jpeg"])
        process_btn = gr.Button("Process Images")
    
    process_status = gr.Textbox(label="Processing Status")
    
    with gr.Tabs():
        with gr.Tab("Camera Poses"):
            pose_btn = gr.Button("Show Camera Poses")
            pose_plot = gr.Plot()
        
        with gr.Tab("Sample Images"):
            sample_btn = gr.Button("Show Images with Axes")
            with gr.Row():
                img1 = gr.Image(label="Sample 1")
                img2 = gr.Image(label="Sample 2")
                img3 = gr.Image(label="Sample 3")
                img4 = gr.Image(label="Sample 4")
                img5 = gr.Image(label="Sample 5")
        
        with gr.Tab("Undistortion"):
            undist_btn = gr.Button("Show Undistortion Preview")
            with gr.Row():
                original = gr.Image(label="Original")
                undistorted = gr.Image(label="Undistorted")
    
    # Event handlers
    process_btn.click(
        workflow.process_images, 
        inputs=[files], 
        outputs=[process_status]
    )
    
    pose_btn.click(
        workflow.get_camera_pose_plot, 
        outputs=[pose_plot]
    )
    
    sample_btn.click(
        workflow.get_sample_images_with_axes, 
        outputs=[img1, img2, img3, img4, img5]
    )
    
    undist_btn.click(
        workflow.get_undistortion_preview, 
        outputs=[original, undistorted]
    )

# Launch interface
app = demo.launch(share=True, debug=False)

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://d8c0ba0f1dc42556e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
